# Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import re
import json
import tqdm

# Data load

In [ ]:
DATA_PATH = '../data/test_data_tomt/'

In [ ]:
with open(DATA_PATH+'sentenceWikiScrappedUpdated.txt', "r") as wiki:
  dataset = json.loads(wiki.read())
dataset

{'Movies': ['Twisted',
  'Chatroom',
  'The 10th Kingdom',
  'The Dragon Ring',
  'Nowhere',
  'Deadend',
  'The Warlord',
  'The Final',
  'Tales of the Night',
  'The Watcher in the Woods'],
 'Plots': [['One evening, the Collins, Phillip and Evelyn, a married couple with two children, discover their maid, Mrs. Murdock, dead at the end of their steps; her neck is broken.',
   'Evidently, she had an accident; now they need a new babysitter for an upcoming party.',
   'The sensible Helen meets little Susan Collins at the discount market and likes her, so she offers to do the job.',
   "She does not know Susan's teenage brother Mark (Christian Slater): technically skilled and good in school, but restive and cunning.",
   'Mark also listens regularly to German marching music from the Third Reich.',
   'As soon as the parents have left, he psychologically terrorizes Helen and his sister with electronic tricks.',
   'Williams (Karl Taylor), a school jock whom Mark burned earlier in science 

In [ ]:
testset = pd.read_csv(DATA_PATH+'queries.csv')
testset

,Query,Movie
0,Teenagers were wandering around some deserted ...,Nowhere
1,one of the characters younger brother (also te...,Nowhere
2,one night the teenagers took the new kid to a ...,Nowhere
3,Someone turns into a giant lizard or some kind...,Nowhere
4,The girl is taking shelter in that tree and wh...,The 10th Kingdom
...,...,...
121,story takes place in a hotel or dormitory buil...,Chatroom
122,story takes place in a hotel or dormitory type...,Chatroom
123,the story takes place in a hotel or dormitory ...,Chatroom
124,story takes place in a hotel or dormitory type...,Chatroom


# BERT

## bert embeddings + cosine

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 29.8 MB/s 
     |████████████████████████████████| 182 kB 67.5 MB/s 
     |████████████████████████████████| 7.6 MB 57.6 MB/s 


In [ ]:
import torch
from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
%matplotlib inline

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def bert_embedding(sentence, model):
  # convert words to tokens
  # make segment_ids 
  # pass to model
  marked_text = "[CLS] " + sentence + " [SEP]"

  # Tokenize our sentence with the BERT tokenizer.
  tokenized_text = tokenizer.tokenize(marked_text)

  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  segments_ids = [1] * len(tokenized_text)

  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])
  model.eval()
  # print(tokens_tensor.shape)

  with torch.no_grad():

      outputs = model(tokens_tensor, segments_tensors)

      # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
      # print(outputs) #contains 'last_hidden_state', 'pooler_output', 'hidden_states'
      hidden_states = outputs[2] #all model weights
      # print(hidden_states.size())
      token_embeddings = torch.stack(hidden_states, dim=0)
      # print(token_embeddings.size())
      token_embeddings = torch.squeeze(token_embeddings, dim=1)
      token_embeddings = token_embeddings.permute(1,0,2) # n_tokens x n_layers (13) x 768

      #sum the last four hidden layers for each word to get word embeddings (experiment here)
      # n_tokens x 768
      word_embeddings = torch.sum(token_embeddings[:,-4:-1,:], dim=1)
      # average all word embeddings to get sentence embeddings
      sentence_embedding=torch.mean(word_embeddings, dim=0) # 768 dim vector

      return sentence_embedding

In [ ]:
dataset['Movies'], dataset['Plots'][0]

(['Twisted',
  'Chatroom',
  'The 10th Kingdom',
  'The Dragon Ring',
  'Nowhere',
  'Deadend',
  'The Warlord',
  'The Final',
  'Tales of the Night',
  'The Watcher in the Woods'],
 ['One evening, the Collins, Phillip and Evelyn, a married couple with two children, discover their maid, Mrs. Murdock, dead at the end of their steps; her neck is broken.',
  'Evidently, she had an accident; now they need a new babysitter for an upcoming party.',
  'The sensible Helen meets little Susan Collins at the discount market and likes her, so she offers to do the job.',
  "She does not know Susan's teenage brother Mark (Christian Slater): technically skilled and good in school, but restive and cunning.",
  'Mark also listens regularly to German marching music from the Third Reich.',
  'As soon as the parents have left, he psychologically terrorizes Helen and his sister with electronic tricks.',
  'Williams (Karl Taylor), a school jock whom Mark burned earlier in science class, comes to the Collin

In [ ]:
# plot embeddings
scene_embeddings = dict()
for i in range(len(dataset['Movies'])):
  print(dataset['Movies'][i])
  embedding = list()
  for scene in tqdm.tqdm(dataset['Plots'][i]):
    embedding.append(bert_embedding(scene, model))
  scene_embeddings[dataset['Movies'][i]] = embedding

Twisted


100%|██████████| 12/12 [00:02<00:00,  4.58it/s]


Chatroom


100%|██████████| 31/31 [00:03<00:00,  7.81it/s]


The 10th Kingdom


...

Tales of the Night


100%|██████████| 109/109 [00:13<00:00,  7.80it/s]


The Watcher in the Woods


100%|██████████| 23/23 [00:04<00:00,  5.46it/s]


AttributeError: ignored

In [ ]:
scene_embeddings["Twisted"][0].shape

torch.Size([768])

In [ ]:
query_embeddings = []

# query embeddings
for i in tqdm.tqdm(range(testset.shape[0])):
    query, movie = testset['Query'][i], testset['Movie'][i]
    query_embeddings.append([bert_embedding(query, model), movie])

100%|██████████| 126/126 [00:12<00:00,  9.89it/s]


In [ ]:
query_embeddings[0][0].shape

torch.Size([768])

In [ ]:
len(query_embeddings)

126

In [ ]:
#top-1  accuracy count
def bertembedding_prediction_top1(scene_embeddings, query_embeddings):
  answer = {"Correct": 0, "Wrong": 0}
  for i in tqdm.tqdm(range(len(query_embeddings))): # embeddings of each test query
    query, actual_movie = query_embeddings[i][0], query_embeddings[i][1]
    print("\n## ", actual_movie)
    maximum = -np.inf
    predicted_movie = ""
    for movie in scene_embeddings: #every movie in our considered movie set
      for scene in scene_embeddings[movie]: #every scene in considered movie
        output = torch.cosine_similarity(query.reshape(1,-1), scene.reshape(1,-1))
        output = float(output[0])
        if output>maximum:
          maximum = output
          predicted_movie = movie
    print("\n++ [", actual_movie, "||", predicted_movie, "]", end=" == ")
    if actual_movie==predicted_movie:
      answer['Correct'] += 1
      print("correct || ", answer)
    else:
      answer['Wrong'] += 1
      print("wrong || ", answer)
    
    
  return answer

top1_predictions_bert = bertembedding_prediction_top1(scene_embeddings, query_embeddings)

  4%|▍         | 5/126 [00:00<00:02, 42.60it/s]


##  Nowhere

++ [ Nowhere || Deadend ] == wrong ||  {'Correct': 0, 'Wrong': 1}

##  Nowhere

++ [ Nowhere || Tales of the Night ] == wrong ||  {'Correct': 0, 'Wrong': 2}

##  Nowhere

++ [ Nowhere || Tales of the Night ] == wrong ||  {'Correct': 0, 'Wrong': 3}

##  Nowhere

++ [ Nowhere || Tales of the Night ] == wrong ||  {'Correct': 0, 'Wrong': 4}

##  The 10th Kingdom

++ [ The 10th Kingdom || Deadend ] == wrong ||  {'Correct': 0, 'Wrong': 5}

##  The 10th Kingdom

++ [ The 10th Kingdom || Nowhere ] == wrong ||  {'Correct': 0, 'Wrong': 6}

##  The 10th Kingdom

++ [ The 10th Kingdom || Tales of the Night ] == wrong ||  {'Correct': 0, 'Wrong': 7}

##  The 10th Kingdom


 11%|█         | 14/126 [00:00<00:02, 38.82it/s]


++ [ The 10th Kingdom || Chatroom ] == wrong ||  {'Correct': 0, 'Wrong': 8}

##  The 10th Kingdom

++ [ The 10th Kingdom || Chatroom ] == wrong ||  {'Correct': 0, 'Wrong': 9}

##  The 10th Kingdom

++ [ The 10th Kingdom || Tales of the Night ] == wrong ||  {'Correct': 0, 'Wrong': 10}

##  The 10th Kingdom

++ [ The 10th Kingdom || Tales of the Night ] == wrong ||  {'Correct': 0, 'Wrong': 11}

##  The 10th Kingdom

++ [ The 10th Kingdom || Tales of the Night ] == wrong ||  {'Correct': 0, 'Wrong': 12}

##  The 10th Kingdom

++ [ The 10th Kingdom || Chatroom ] == wrong ||  {'Correct': 0, 'Wrong': 13}

##  The 10th Kingdom

++ [ The 10th Kingdom || Tales of the Night ] == wrong ||  {'Correct': 0, 'Wrong': 14}

##  The Dragon Ring

++ [ The Dragon Ring || Tales of the Night ] == wrong ||  {'Correct': 0, 'Wrong': 15}

##  The Dragon Ring

++ [ The Dragon Ring || The Dragon Ring ] == correct ||  {'Correct': 1, 'Wrong': 15}

## 

 19%|█▉        | 24/126 [00:00<00:02, 40.33it/s]

...


++ [ The Watcher in the Woods || Nowhere ] == wrong ||  {'Correct': 38, 'Wrong': 68}

##  The Watcher in the Woods

++ [ The Watcher in the Woods || Deadend ] == wrong ||  {'Correct': 38, 'Wrong': 69}

##  Nowhere

++ [ Nowhere || Chatroom ] == wrong ||  {'Correct': 38, 'Wrong': 70}

##  Nowhere

++ [ Nowhere || Tales of the Night ] == wrong ||  {'Correct': 38, 'Wrong': 71}

##  Nowhere

++ [ Nowhere || Tales of the Night ] == wrong ||  {'Correct': 38, 'Wrong': 72}

##  Nowhere

++ [ Nowhere || Tales of the Night ] == wrong ||  {'Correct': 38, 'Wrong': 73}

##  Chatroom

++ [ Chatroom || Tales of the Night ] == wrong ||  {'Correct': 38, 'Wrong': 74}

##  Chatroom

++ [ Chatroom || Deadend ] == wrong ||  {'Correct': 38, 'Wrong': 75}

##  Chatroom


 94%|█████████▍| 119/126 [00:03<00:00, 38.39it/s]


++ [ Chatroom || Chatroom ] == correct ||  {'Correct': 39, 'Wrong': 75}

##  Chatroom

++ [ Chatroom || Tales of the Night ] == wrong ||  {'Correct': 39, 'Wrong': 76}

##  Chatroom

++ [ Chatroom || Tales of the Night ] == wrong ||  {'Correct': 39, 'Wrong': 77}

##  Chatroom

++ [ Chatroom || Tales of the Night ] == wrong ||  {'Correct': 39, 'Wrong': 78}

##  Chatroom

++ [ Chatroom || Chatroom ] == correct ||  {'Correct': 40, 'Wrong': 78}

##  Chatroom

++ [ Chatroom || Deadend ] == wrong ||  {'Correct': 40, 'Wrong': 79}

##  Chatroom

++ [ Chatroom || Deadend ] == wrong ||  {'Correct': 40, 'Wrong': 80}

##  Chatroom

++ [ Chatroom || Nowhere ] == wrong ||  {'Correct': 40, 'Wrong': 81}

##  Chatroom


100%|██████████| 126/126 [00:03<00:00, 38.63it/s]


++ [ Chatroom || Chatroom ] == correct ||  {'Correct': 41, 'Wrong': 81}

##  Chatroom

++ [ Chatroom || Chatroom ] == correct ||  {'Correct': 42, 'Wrong': 81}

##  Chatroom

++ [ Chatroom || Chatroom ] == correct ||  {'Correct': 43, 'Wrong': 81}

##  Chatroom

++ [ Chatroom || Chatroom ] == correct ||  {'Correct': 44, 'Wrong': 81}

##  Chatroom

++ [ Chatroom || Chatroom ] == correct ||  {'Correct': 45, 'Wrong': 81}


In [ ]:
top1_predictions_bert

{'Correct': 45, 'Wrong': 81}

In [ ]:
top1_predictions_bert['Correct']/(top1_predictions_bert['Correct']+top1_predictions_bert['Wrong'])

0.35714285714285715

In [ ]:
#top-3  accuracy count
def bertembedding_prediction_top3(scene_embeddings, query_embeddings):
  answer = {"Correct": 0, "Wrong": 0}
  for i in tqdm.tqdm(range(len(query_embeddings))): # embeddings of each test query
    query, actual_movie = query_embeddings[i][0], query_embeddings[i][1]
    print("\n## ", actual_movie)
    maximum = -np.inf
    predicted_movie = ""
    rank_dict = dict()
    for movie in scene_embeddings: #every movie in our considered movie set
      maximum = -np.inf
      for scene in scene_embeddings[movie]: #every scene in considered movie
        output = torch.cosine_similarity(query.reshape(1,-1), scene.reshape(1,-1))
        output = float(output[0])
        if output>maximum:
          maximum = output
          predicted_movie = movie
      rank_dict[predicted_movie] = maximum
    rank_dict_top3=set( movie.lower().strip() for (movie, score) in list(sorted(rank_dict.items(), key=lambda item: item[1], reverse = True))[:3])
    print("\n++ [", actual_movie, "||", rank_dict_top3, "]", end=" == ")
    if actual_movie.lower().strip() in rank_dict_top3:
      answer['Correct'] += 1
      print("correct || ", answer)
    else:
      answer['Wrong'] += 1
      print("wrong || ", answer)
    
    
  return answer

top3_predictions_bert = bertembedding_prediction_top3(scene_embeddings, query_embeddings)

  3%|▎         | 4/126 [00:00<00:03, 36.76it/s]


##  Nowhere

++ [ Nowhere || {'tales of the night', 'nowhere', 'deadend'} ] == correct ||  {'Correct': 1, 'Wrong': 0}

##  Nowhere

++ [ Nowhere || {'tales of the night', 'nowhere', 'deadend'} ] == correct ||  {'Correct': 2, 'Wrong': 0}

##  Nowhere

++ [ Nowhere || {'tales of the night', 'chatroom', 'nowhere'} ] == correct ||  {'Correct': 3, 'Wrong': 0}

##  Nowhere

++ [ Nowhere || {'tales of the night', 'nowhere', 'the watcher in the woods'} ] == correct ||  {'Correct': 4, 'Wrong': 0}

##  The 10th Kingdom

++ [ The 10th Kingdom || {'tales of the night', 'nowhere', 'deadend'} ] == wrong ||  {'Correct': 4, 'Wrong': 1}

##  The 10th Kingdom

++ [ The 10th Kingdom || {'nowhere', 'chatroom', 'deadend'} ] == wrong ||  {'Correct': 4, 'Wrong': 2}

##  The 10th Kingdom

++ [ The 10th Kingdom || {'tales of the night', 'nowhere', 'chatroom'} ] == wrong ||  {'Correct': 4, 'Wrong': 3}

##  The 10th Kingdom


 10%|▉         | 12/126 [00:00<00:03, 35.15it/s]


++ [ The 10th Kingdom || {'the warlord', 'chatroom', 'tales of the night'} ] == wrong ||  {'Correct': 4, 'Wrong': 4}

##  The 10th Kingdom

++ [ The 10th Kingdom || {'chatroom', 'the dragon ring', 'deadend'} ] == wrong ||  {'Correct': 4, 'Wrong': 5}

##  The 10th Kingdom

++ [ The 10th Kingdom || {'twisted', 'tales of the night', 'deadend'} ] == wrong ||  {'Correct': 4, 'Wrong': 6}

##  The 10th Kingdom

++ [ The 10th Kingdom || {'tales of the night', 'deadend', 'the 10th kingdom'} ] == correct ||  {'Correct': 5, 'Wrong': 6}

##  The 10th Kingdom

++ [ The 10th Kingdom || {'tales of the night', 'the watcher in the woods', 'the 10th kingdom'} ] == correct ||  {'Correct': 6, 'Wrong': 6}

##  The 10th Kingdom

++ [ The 10th Kingdom || {'tales of the night', 'chatroom', 'deadend'} ] == wrong ||  {'Correct': 6, 'Wrong': 7}

##  The 10th Kingdom

++ [ The 10th Kingdom || {'the warlord', 'tales of the night', 'the 10th kingdom'} ] == correct ||  {'Correct': 7, 'Wrong': 7}

##  The Dragon Rin

 16%|█▌        | 20/126 [00:00<00:02, 35.69it/s]

...


++ [ Nowhere || {'the warlord', 'tales of the night', 'chatroom'} ] == wrong ||  {'Correct': 71, 'Wrong': 38}

##  Nowhere

++ [ Nowhere || {'tales of the night', 'nowhere', 'the watcher in the woods'} ] == correct ||  {'Correct': 72, 'Wrong': 38}

##  Nowhere

++ [ Nowhere || {'tales of the night', 'nowhere', 'the watcher in the woods'} ] == correct ||  {'Correct': 73, 'Wrong': 38}

##  Chatroom

++ [ Chatroom || {'the warlord', 'tales of the night', 'chatroom'} ] == correct ||  {'Correct': 74, 'Wrong': 38}

##  Chatroom

++ [ Chatroom || {'the warlord', 'deadend', 'the 10th kingdom'} ] == wrong ||  {'Correct': 74, 'Wrong': 39}

##  Chatroom

++ [ Chatroom || {'deadend', 'chatroom', 'twisted'} ] == correct ||  {'Correct': 75, 'Wrong': 39}

##  Chatroom

++ [ Chatroom || {'tales of the night', 'deadend', 'the 10th kingdom'} ] == wrong ||  {'Correct': 75, 'Wrong': 40}

##  Chatroom

++ [ Chatroom || {'tales of the night', 'deadend', 'the 10th kingdom'} ] == wrong ||  {'Correct': 75, 'W

 99%|█████████▉| 125/126 [00:03<00:00, 40.14it/s]

 Chatroom || {'tales of the night', 'deadend', 'the 10th kingdom'} ] == wrong ||  {'Correct': 75, 'Wrong': 42}

##  Chatroom

++ [ Chatroom || {'chatroom', 'deadend', 'the 10th kingdom'} ] == correct ||  {'Correct': 76, 'Wrong': 42}

##  Chatroom

++ [ Chatroom || {'chatroom', 'deadend', 'nowhere'} ] == correct ||  {'Correct': 77, 'Wrong': 42}

##  Chatroom

++ [ Chatroom || {'chatroom', 'deadend', 'nowhere'} ] == correct ||  {'Correct': 78, 'Wrong': 42}

##  Chatroom

++ [ Chatroom || {'nowhere', 'chatroom', 'deadend'} ] == correct ||  {'Correct': 79, 'Wrong': 42}

##  Chatroom

++ [ Chatroom || {'the warlord', 'tales of the night', 'chatroom'} ] == correct ||  {'Correct': 80, 'Wrong': 42}

##  Chatroom

++ [ Chatroom || {'the warlord', 'tales of the night', 'chatroom'} ] == correct ||  {'Correct': 81, 'Wrong': 42}

##  Chatroom

++ [ Chatroom || {'the warlord', 'tales of the night', 'chatroom'} ] == correct ||  {'Correct': 82, 'Wrong': 42}

##  Chatroom

++ [ Chatroom || {'the warlor

100%|██████████| 126/126 [00:03<00:00, 39.39it/s]


++ [ Chatroom || {'the warlord', 'tales of the night', 'chatroom'} ] == correct ||  {'Correct': 84, 'Wrong': 42}


In [ ]:
top3_predictions_bert

{'Correct': 84, 'Wrong': 42}

In [ ]:
top3_predictions_bert['Correct']/(top3_predictions_bert['Correct']+top3_predictions_bert['Wrong'])

0.6666666666666666

## pip installs for bertscore

In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 1.2 MB/s 
     |████████████████████████████████| 115 kB 64.8 MB/s 
     |████████████████████████████████| 182 kB 69.8 MB/s 
     |████████████████████████████████| 451 kB 61.9 MB/s 
     |████████████████████████████████| 212 kB 70.7 MB/s 
     |████████████████████████████████| 127 kB 71.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60 kB 9.0 MB/s 
     |████████████████████████████████| 5.5 MB 52.3 MB/s 
     |████████████████████████████████| 7.6 MB 57.4 MB/s 


In [ ]:
from evaluate import load
bertscore = load("bertscore")
predictions = ["hello world", "general kenobi"]
references = ["goodnight moon"]*len(predictions) #, "the sun is shining"]
results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")
print(results)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

{'precision': [0.789967954158783, 0.5812705755233765], 'recall': [0.789967954158783, 0.6331430077552795], 'f1': [0.789967954158783, 0.6060988903045654], 'hashcode': 'distilbert-base-uncased_L5_no-idf_version=0.3.12(hug_trans=4.24.0)'}


In [ ]:
results['f1'][0]

0.789967954158783

## bertscore
ref: https://huggingface.co/spaces/evaluate-metric/bertscore

In [ ]:
#top-1  accuracy count
def bertscore_prediction_top1(testset, dataset):
  answer = {"Correct": 0, "Wrong": 0}
  for i in tqdm.tqdm(range(testset.shape[0])): # embeddings of each test query
    query, actual_movie = testset['Query'][i], testset['Movie'][i]
    print("\n## ", query, actual_movie)
    maximum = -np.inf
    predicted_movie = ""
    for movie, plots in zip(dataset['Movies'], dataset['Plots']):
      print("\t\t\t\t\t\t\t\t", movie)
      for scene in plots:
        output = bertscore.compute(predictions=[scene], references=[query], model_type="distilbert-base-uncased") #can eliminate one for loop here and pass all scenes directly
        output = float(output['f1'][0])
        if output>maximum:
          maximum = output
          predicted_movie = movie
    print("\n++ [", actual_movie, "||", predicted_movie, "]", end=" == ")
    if actual_movie.lower().strip()==predicted_movie.lower().strip():
      answer['Correct'] += 1
      print("correct || ", answer)
    else:
      answer['Wrong'] += 1
      print("wrong || ", answer)
    
    
  return answer

top1_predictions = bertscore_prediction_top1(testset, dataset)


  0%|          | 0/126 [00:00<?, ?it/s]


##  Teenagers were wandering around some deserted areas with lots of graffitis and deserted buildings, in one of which there was a club (?) Nowhere
								 Twisted
								 Chatroom
								 The 10th Kingdom
								 The Dragon Ring
								 Nowhere
								 Deadend
								 The Warlord
								 The Final
								 Tales of the Night
								 The Watcher in the Woods


  1%|          | 1/126 [00:05<12:09,  5.84s/it]


++ [ Nowhere || Tales of the Night ] == wrong ||  {'Correct': 0, 'Wrong': 1}

##  one of the characters younger brother (also teenager) with his girlfriend drove a red cadillack to some hill during the night on one of their dates, I remember they were really fucked up psychos (can’t remember why, maybe they killed someone) 6. Nowhere
								 Twisted
								 Chatroom
								 The 10th Kingdom
								 The Dragon Ring
								 Nowhere
								 Deadend
								 The Warlord
								 The Final
								 Tales of the Night
								 The Watcher in the Woods


  2%|▏         | 2/126 [00:10<11:00,  5.32s/it]

...


++ [ Chatroom || The Warlord ] == wrong ||  {'Correct': 47, 'Wrong': 77}

##  story takes place in a hotel or dormitory type building where the doors are numbered Chatroom
								 Twisted
								 Chatroom
								 The 10th Kingdom
								 The Dragon Ring
								 Nowhere
								 Deadend
								 The Warlord
								 The Final
								 Tales of the Night
								 The Watcher in the Woods


 99%|█████████▉| 125/126 [10:12<00:05,  5.14s/it]


++ [ Chatroom || The Warlord ] == wrong ||  {'Correct': 47, 'Wrong': 78}

##  the story takes place in a hotel or dormitory type building where the doors are numbered Chatroom
								 Twisted
								 Chatroom
								 The 10th Kingdom
								 The Dragon Ring
								 Nowhere
								 Deadend
								 The Warlord
								 The Final
								 Tales of the Night
								 The Watcher in the Woods


100%|██████████| 126/126 [10:17<00:00,  4.90s/it]


++ [ Chatroom || The Warlord ] == wrong ||  {'Correct': 47, 'Wrong': 79}


In [ ]:
top1_predictions

{'Correct': 47, 'Wrong': 79}

In [ ]:
top1_predictions['Correct']/(top1_predictions['Correct']+top1_predictions['Wrong'])

0.373015873015873

In [ ]:
#top-3 accuracy count
def bertscore_prediction_top3(testset, dataset):
  answer = {"Correct": 0, "Wrong": 0}
  for i in tqdm.tqdm(range(testset.shape[0])): # embeddings of each test query
    query, actual_movie = testset['Query'][i], testset['Movie'][i]
    print("\n## ", query, actual_movie)
    query_sentence = ""
    maximum = -np.inf
    rank_dict=dict()
    predicted_movie = ""
    for movie, plots in zip(dataset['Movies'], dataset['Plots']):
      print("\t\t\t\t\t\t\t\t", movie)
      for scene in plots:
        output = bertscore.compute(predictions=[scene], references=[query], model_type="distilbert-base-uncased") #can eliminate one for loop here and pass all scenes directly
        output = float(output['f1'][0])
        if output>maximum:
          maximum = output
          predicted_movie = movie
          query_sentence = scene
      rank_dict[predicted_movie] = [maximum, query_sentence]
    rank_dict_top3=dict({movie.lower().strip(): [score, scene] for (movie, (score, scene)) in list(sorted(rank_dict.items(), key=lambda item: item[1], reverse = True))[:3]})
    print("\n++ [", actual_movie, "||", rank_dict_top3, "]")
    if actual_movie.lower().strip() in rank_dict_top3:
      answer['Correct'] += 1
      print("correct || ", answer)
    else:
      answer['Wrong'] += 1
      print("wrong || ", answer)
    
  return answer

top3_predictions = bertscore_prediction_top3(testset, dataset)


  0%|          | 0/126 [00:00<?, ?it/s]


##  Teenagers were wandering around some deserted areas with lots of graffitis and deserted buildings, in one of which there was a club (?) Nowhere
								 Twisted
								 Chatroom
								 The 10th Kingdom
								 The Dragon Ring
								 Nowhere
								 Deadend
								 The Warlord
								 The Final
								 Tales of the Night
								 The Watcher in the Woods


  1%|          | 1/126 [00:05<12:23,  5.95s/it]


++ [ Nowhere || {'tales of the night': [0.7378039956092834, 'A girl, a boy and an old cinema technician tell stories every night in a small theater.'], 'the final': [0.7347840070724487, 'Among their inspirations are horror films and lessons in class about deadly chemicals and torture methods of old cultures.'], 'deadend': [0.7297780513763428, 'She begins to walk and sees body bags containing her dead family members in the middle of the road.']} ]
wrong ||  {'Correct': 0, 'Wrong': 1}

##  one of the characters younger brother (also teenager) with his girlfriend drove a red cadillack to some hill during the night on one of their dates, I remember they were really fucked up psychos (can’t remember why, maybe they killed someone) 6. Nowhere
								 Twisted
								 Chatroom
								 The 10th Kingdom
								 The Dragon Ring
								 Nowhere
								 Deadend
								 The Warlord
								 The Final
								 Tales of the Night
								 The Watcher in the Woods


  2%|▏         | 2/126 [00:11<11:17,  5.46s/it]

...


++ [ Chatroom || {'the warlord': [0.8050935864448547, 'The movie opens in the living room of a very luxurious house.'], 'chatroom': [0.7757170796394348, 'Even though they only really communicate through text, they are depicted in an old hotel-like room and actually having contact.'], 'twisted': [0.6906159520149231, 'The sensible Helen meets little Susan Collins at the discount market and likes her, so she offers to do the job.']} ]
correct ||  {'Correct': 69, 'Wrong': 55}

##  story takes place in a hotel or dormitory type building where the doors are numbered Chatroom
								 Twisted
								 Chatroom
								 The 10th Kingdom
								 The Dragon Ring
								 Nowhere
								 Deadend
								 The Warlord
								 The Final
								 Tales of the Night
								 The Watcher in the Woods


 99%|█████████▉| 125/126 [10:10<00:04,  4.87s/it]


++ [ Chatroom || {'the warlord': [0.7876343727111816, 'The movie opens in the living room of a very luxurious house.'], 'chatroom': [0.7752012014389038, 'Even though they only really communicate through text, they are depicted in an old hotel-like room and actually having contact.'], 'twisted': [0.6942894458770752, 'The sensible Helen meets little Susan Collins at the discount market and likes her, so she offers to do the job.']} ]
correct ||  {'Correct': 70, 'Wrong': 55}

##  the story takes place in a hotel or dormitory type building where the doors are numbered Chatroom
								 Twisted
								 Chatroom
								 The 10th Kingdom
								 The Dragon Ring
								 Nowhere
								 Deadend
								 The Warlord
								 The Final
								 Tales of the Night
								 The Watcher in the Woods


100%|██████████| 126/126 [10:15<00:00,  4.89s/it]


++ [ Chatroom || {'the warlord': [0.8017410635948181, 'The movie opens in the living room of a very luxurious house.'], 'chatroom': [0.7749226689338684, 'Even though they only really communicate through text, they are depicted in an old hotel-like room and actually having contact.'], 'twisted': [0.696723461151123, 'The sensible Helen meets little Susan Collins at the discount market and likes her, so she offers to do the job.']} ]
correct ||  {'Correct': 71, 'Wrong': 55}


In [ ]:
top3_predictions

{'Correct': 71, 'Wrong': 55}

In [ ]:
top3_predictions['Correct']/(top3_predictions['Correct']+top3_predictions['Wrong'])

0.5634920634920635